In [23]:
# -*- coding: utf-8 -*-
"""
Created on Wed Dec 22 17:12:01 2021

@author: User
"""
# import 爬蟲所需要的東西
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import numpy as np
import os,sys,time
from PIL import Image
from urllib import request
import requests
import json
import sys
import modin.pandas as pd
from iteration_utilities import duplicates
import asyncio
import unittest
from numba import jit
import csv
import copy
import networkx as nx


# P.S.
# python 預設調用  print， 會調用 __str__ 函數。
# __rerp__ 是預覽



# 實作　Node 單位


class Node:
    #建構子，初始化節點
    #初始authority跟 hub,值皆為1
    def __init__(self, ID): #一定會有self這個值，並用self代稱上面的class_name(Node)
        self.ID = ID
        self.parents = []
        self.children = []  #初始的children 跟 parents串列裡面都是空的。(因為都沒有建立任何連結)
        self.auth = 1.0
        self.hub = 1.0
    # __str__ 是print會調用的函數
    def __str__(self):
        return str(self.ID)
        
    def show(self): #所有的function裡面都要有self，代指使用自己方法的本身。
        print(f"This's {self.ID}")
        
    #link_child 跟 link_parent，為網站間的有向連結，但其並沒有考慮到各child節點或各parent節點之間加入到"children"或"parent"串列的先後順序。
    #只能計算此節點所有連出的節點個數計算。
        
    def link_child(self,new_child): #這Method中有兩個參數，self如上述，"new child" 為其他的Node類物件，近來這個程式的代稱為"new_child"
        for child in self.children: #在呼叫Method link_child的 Node 中，依序搜尋children 串列。
            if(new_child.ID == child.ID): #若已經有了，回傳None
                print("duplicate") #若有重複印出"duplicate"
                #print(child.ID)
                return None
        self.children.append(new_child)
        
    def link_parent(self,new_parent): #同上
        for parent in self.parents:
            if (new_parent.ID == self.ID):
                print("duplicate")
                #print(self.ID)
                return None
        self.parents.append(new_parent)
        
    def show_children(self): #印出一個節點的所有孩子(印出這個node所有連出的連結)
        if len(self.children) == 0:
            print("this node :",self.ID,", children_list is empty.")
        else:
            for i in range(len(self.children)):
                print(self.children[i].ID)
        
    def show_parents(self): #印出一個節點的所有parent(印出這個node所有連入的連結)
        for i in range(len(self.parents)):
            print(self.parents[i].ID)
            
    def update_hub(self):  #更新此點的hub分數
        hub_score = 1
        for i in self.children:
            hub_score += i.hub
        return hub_score
    
    def update_auth(self):   #更新此點的authority分數
        authority_score = 0
        for i in self.parents:
            authority_score += i.auth
        return authority_score
    
    def show_indegree(self):  #算出所有此點的 in_degree
        pass
    
    def show_outdegree(self):  #算出所有此點的 out_degree
        pass
    
    def normalize_hub_score(self):   #將此點的hub分數除以全部的hub分數總合
        pass
    
    def normalize_authority_socre(self):   #將此點的authority分數除已全部的authority分數總合。
        pass
    
#實作 Graph

class Graph:   #在這個網絡上(圖)的所有網站(點)集合
    def __init__(self, G_name):
        self.name = G_name  #圖的名字
        self.nodes = [] #一開始的nodes集合為空
        self.edges = [] #所有的邊集合我也設一個串列存起來，但這邊我目前還沒有想到怎麼加入可以加入有向邊。
    
    def insert_node(self, node):  #將外來的點加入這個圖的點集合中
        try:    
            self.nodes.append(node)
            # print("successfully insert！")
        except error as e:
            print(f"insert fail:{e}")
    
    def display_hub_authority_score(self):
        pass
    
    def find_node(self, ID):  # 要找的點的名字，也就是網址
        for node in self.nodes: # 在自己圖中的節點做尋找
            if node.ID == ID:
                # print("found")
                return node # 找到的話就回傳節點
        else: # 否則就說沒找到
            # print("Not found")
            return None # 回傳空值

# 將所有 ID 一樣的都刪除。
def set_node_list(list_a):
    # 比較所有的ID名稱，先用for迴圈將所有的ID名稱存起來。
    all_name = [] #所有名字的列表
    for i in list_a:
        all_name.append(i.ID)
    all_name_count = [] #計算每個網站重複的數量
    for index,i in enumerate(list_a):  #對每個list_a 的元素做檢查
        count = 0
        for k in all_name: # 在所有的名字中搜尋，如果有一樣的名字，count就加一。
            if i.ID == k:
                count +=1
        all_name_count.append(count)
    for index,i in enumerate(all_name_count):  #超過一次以上的就刪除
        if i > 1:
            del list_a[index]
            del all_name[index]
            del all_name_count[index]
    return list_a                   #回傳一list 其為不重複的 node版本

def search_all_link(query_link):
    a = []  # 這個網站所有的 outlink
    html = requests.get(query_link, timeout = 10)
    # 為防止爬蟲卡死情況(request 不會自行斷開)，故上面設置 timeout 參數，設定固定時間若未完成則 斷開連結。
    html.encoding = 'UTF-8' #使用html編碼。
    sp = BeautifulSoup(html.text, 'html5lib') #指定'html5lib'作為解析器
    #若只使用sp.link 則只會找出第一筆
    #故這邊要使用"find.all"語法
    link_search = sp.find_all("a") #找出所有的 <a> (超連結)
    for i in range(len(link_search)):
        k = str(link_search[i].get("href"))  #這行一定要記得要轉type！！！轉成str不然有時候會無法切片取值。
        if k[0:5] == "https": #只留下"https"
            a.append(k)
    return a  #回傳一列網址(type list)

#先將所有node_list中的資料轉換成type str存入到list再寫入檔案
def node_to_text(all_node_list):
    node_list_txt = [] # node_list 的所有內容
    for i in all_node_list:
        tmp_list = [] # 初始化
        # 依序記錄結構化的node的每個資料
        tmp_list.append(i.ID)
        # 這邊嘗試寫遞迴函式，不停往下找
        # 加入cildren
        tmp_children = []
        for j in i.children:
            tmp_children.append(j.ID)
        tmp_list.append(tmp_children)
        # 加入parents
        tmp_parents = []
        for k in i.parents:
            tmp_parents.append(k.ID)
        tmp_list.append(tmp_parents)
        tmp_list.append(i.auth)
        tmp_list.append(i.hub)
        # 再加入list
        node_list_txt.append(tmp_list)
    return node_list_txt  # 回傳一個 list
def listDups(tmp_list_aaa): # 記錄重複的功能
    return list(duplicates(tmp_list_aaa))
def str_to_url(aaa): # aaa 為文字格式。
    bbb = aaa.split(",") # 利用逗號做切分 # bbb為list格式 # 會回傳一個list
    flag_head = False # 一開始起頭開關為False
    tmp_word_list = [] # 裝暫存要轉變成文字的網址，每次只會存一個網址，之後就會被清空
    tmp_url_list = [] # 裝整理好的網址
    for url in bbb: # 在這bbb串列中做遞迴
        for char in url:
            if char == "'": # 當遇見"單引號"時switch (此時Flag_head只有兩種狀態)
                flag_head = not(flag_head) # 轉換開關
                # tmp_word_list.append(char) # 將其收錄至暫存的字串列。
            if flag_head == True:
                tmp_word_list.append(char) # 因為只有h會將其打開，所以一路收錄char 至 tmp_word_list
            if flag_head == False:
                StrA = "".join(tmp_word_list)
                StrA = StrA[1:]  # 去掉前端的單引號。 
                if len(tmp_word_list) != 0: # 不為空才加入
                    tmp_url_list.append(StrA)
                tmp_word_list = [] # 清空tmp_word_list
    # 所以我只要再把這些網址NODE化之後再加入CHILDREN串列就好 =)
    return tmp_url_list #回傳整理好的網址

# 111/1/1發現這邊有問題，雖然建立的nodes名字都一樣，但是因為是從不同檔案load進來的，會有問題。
# 111/1/1目前解決方法，隨然load進來的是不同檔案，但是會load到同個圖中。
def load_node_from_csv_out_side(start_num, end_num, df_name): # 讀csv檔的資料 # 並且加入子節點 # 也同時加入父節點
    # 參數分別為，load加入的檔案的第幾筆到第幾筆，讀入黨案名稱。
# 每個點依序建立連結
    # 在這邊才讀取檔案是為了要避免讀取全域變數
    df = pd.read_csv(df_name)
    start_time = time.time()  #開始執行時間
    for i in range(start_num, end_num + 1): # 先存1-9000筆
        bbb = df["2"][i] # 將一整串str去頭去尾。
        url_list = str_to_url(bbb)  # 會回傳一個整理好的串列
        tmp2 = Graph_all.nodes[i] # 選取第幾個node
        for j in url_list: # 對找到的每個網站
            # 要先觀察 "j" 這個網站名字，是不是已經存在在原有的193721筆資料中。
            a = Graph_all.find_node(j)
            if a != None: # 有找到值，代表此網站已經在圖中了。此時 a 就已經是圖中的節點。 type == node
                tmp2.link_child(a)
                a.link_parent(tmp2)
            else: # 沒找到值，代表這是新的點
                tmp_node = Node(j) # 先將網站節點化
                Graph_all.insert_node(tmp_node) # 圖中也要加入這個新的點。
                Graph_all.nodes[i].link_child(tmp_node) # 加入node化的子節點。 # 這邊確認沒有重複。
                tmp_node.link_parent(tmp2) # 連結原節點。
    print("花費:" + str(float(time.time() - start_time)) + "秒")
    print("finish")
def Diff(li1, li2): 
    return (list(set(li1).symmetric_difference(set(li2))))  
# 判別兩列表不同元素
def diff(list_a, list_b):
    list_1 = list_a.copy()
    list_2 = list_b.copy()
    intersection = [x for x in list_1 for y in list_2 if x == y]
    return intersection

def load_out_link(start, end, csv_file_name): # 讀所有out_link # 回傳一列表
    
    df = pd.read_csv(csv_file_name)
    return_list = [] # 要回傳的列表
    for index in range(start,end+1): # 將前9000筆資料加入列表中
        bbb = df["2"][index]
        url_list = str_to_url(bbb)
        for j in url_list:
            if j[0:4] == "http": # 等於http的才加入。
                #print(j) # 撿查
                return_list.append(j) # 依序加入
    return return_list
    
# 實行二分搜尋
# 定義：在排序好的數列裡，找某 target，找到就回傳其 index，否則回傳 -1
def binary_search(data_list, key_data):  #要加入的圖的點的列表(已知且已排序), 要尋找的 資料
    #設置選取範圍的指標
    low = 0
    upper = len(data_list) - 1
    while low <= upper:
        mid = int((low + upper) / 2) #取中間索引的值
        if data_list[mid] < key_data:    #若搜尋值比中間的值大，將中間索引+1，取右半
            low = mid + 1
        elif data_list[mid] > key_data:  #若搜尋值比中間的值小，將中間索引+1，取左半
            upper = mid - 1
        else:                    #若搜尋值等於中間的值，則回傳
            return mid # 回傳的是索引值。
    return "Node Not Found!"


#  這邊一定要針對外部的圖進行修改，無法不用全域變數。
#  這邊其實已經解決先後順序的問題了，所以可以嘗試看看非同步的寫法。
#  此函式適用於所有load進來的檔案中已經沒有新node(所有的不重複網站名稱都已經存在在以建立之圖中)
#  之後應該要建立一另外之函式for 新加入的網站(以上述的csv檔案儲存)
def build_module_all_nodes_found(original_url, graph, out_link_list_http): 
    # 確定全部的點都加入了才可以使用此函式
    # 參數 原始的網站，要加入的圖(裡面的點已排序)(對這個圖進行操作)，out_link列表
        # 找到原始網站的那個點
        # 0.22s
        # 這邊改用 binary search
    All_graph_nodes_list = graph.nodes # 使用那個圖的點 #  指的是同一個位置 # 這個圖已經排序了
    key_1 = binary_search(All_graph_nodes_list, original_url)
    node_tmp = All_graph_nodes_list[key_1] 
   # print("網站",node_tmp.ID)
    for out_url in out_link_list_http:
        if len(out_link_list_http) != 0:
            key_2 = binary_search(All_graph_nodes_list, out_url)
            # 0.22s
            if key_2 != "Node Not Found!":
                # 利用指標操作找到那個點
                node_tmp_2 = All_graph_nodes_list[key_2] 
                node_tmp.link_child(node_tmp_2)
                node_tmp_2.link_parent(node_tmp)
             #   print("out_link", node_tmp_2.ID)
        else:
            print("empty_outlink")
def binary_search_for_nx(data_list, key_data):  #要加入的圖的點的列表(已知且已排序), 要尋找的 資料
    #設置選取範圍的指標
    low = 0
    upper = len(data_list) - 1
    while low <= upper:
        mid = int((low + upper) / 2) #取中間索引的值
        if data_list[mid] < key_data:    #若搜尋值比中間的值大，將中間索引+1，取右半
            low = mid + 1
        elif data_list[mid] > key_data:  #若搜尋值比中間的值小，將中間索引+1，取左半
            upper = mid - 1
        else:                    #若搜尋值等於中間的值，則回傳
            return mid # 回傳的是索引值。
    return "Node Not Found!"

def top_n_pagerank_for_nx(p, graph, n):
    L = sorted(p.items(), key = lambda item:item[1], reverse=True)
    # 所以上面這一行會做出一個list包著tuple如下(根據value值排列)。
    num = 0
    for i in range(n):
        print("Top", num+1, graph.nodes[L[i][0]])
        print("weight:", L[i][1])
        num += 1
def top_n_authority_for_nx(a, graph, n): 
    # a 為一個dictionary物件，有網站代碼跟其權重值 EX:(1294317, 0.0028239679462683694)
    # grpah 為 nx 套件的圖
    # n: 找前幾名 (int)
    L = sorted(a.items(), key = lambda item:item[1], reverse=True)
    # 所以上面這一行會做出一個list包著tuple如下(根據value值排列)。
    num = 0
    for i in range(n):
        print("Top", num+1, graph.nodes[L[i][0]])
        print("weight:", L[i][1])
        num += 1
def top_n_hub_for_nx(h, graph, n):
    L = sorted(h.items(), key = lambda item:item[1], reverse=True)
    # 所以上面這一行會做出一個list包著tuple如下(根據value值排列)。
    num = 0
    for i in range(n):
        print("Top", num+1, graph.nodes[L[i][0]])
        print("weight:", L[i][1])
        num += 1

In [24]:
# 此城市適用於已經有圖以及其所有的關係！！！！！！
# 此程式用於將已建立之圖，去除其相同之網域，令其從相同網域指入之連結，"禁止連入"！

In [25]:
# 步驟一：要先讀出目標網域，規則：從https開始讀取到 第一個 "/" 為止。
# 步驟二：如果網域相同，那就將網域相同的網頁刪除(parent ,跟 children的網頁都要刪掉)
# 經過此處理，會將所有相同網域的網站都除去，只留下不同網域的連結。

In [114]:
# 測試正則化處理之後抓出的domain
csv.field_size_limit(500 * 1024 * 1024)
csv_file_name = "Graph0308.csv" # 這邊放入原始圖
csvfile = open(csv_file_name, newline='', encoding = "utf-8")
# 讀取 CSV 檔案內容
rows = csv.reader(csvfile, delimiter=';') # 設定分隔號
# 設定正則表達式模型
original_domain_md = re.compile(
    '(([a-zA-Z0-9\._-]+\.[a-zA-Z]{2,100})|([0-9]{1,100}|[\w]{2,100}\.[\d]{1,100}|[\d]{2,100}\.[\d]{1,100}|[\w]{2,100}\.[\d]{1,100}))'
)
# 以迴圈檢查檢查前11筆
#count = 0
all_node = []
for row in rows:
    node = []
    #if count == 11:
        #break
    #print('original',row[1]) # <class 'str'> # original url
    if row[1][:8] != "https://":
        continue # 跳過不儲存
    original_domain = original_domain_md.search(row[1])
    # if original_domain != None:
        # print(original_domain.group()) # <class "str">
    #檢查所有的 link
    # print(row[2]) # <class 'str'> # parent list
    parents_list = str_to_url(row[2])
    # print(type(parents_list))
    # 刪除domain相同的parent
    for index,parent in enumerate(parents_list):
        if parent[:8] != "https://":
            del parents_list[index]
        else:
            url_domain = original_domain_md.search(parent)
            try:
                if url_domain.group() == original_domain.group():
                    del parents_list[index]
            except Exception as e:
                pass
    # print(row[3]) # <class 'str'> # children list
    children_list = str_to_url(row[3])
    # 刪除domain相同的child
    for index2,child in enumerate(children_list):
        if child[:8] != "https://":
            del children_list[index2]
        else:    
            url_domain2 = original_domain_md.search(child)
            #print(url_domain.group())
            try:
                if url_domain2.group() == original_domain.group():
                    del children_list[index2]
            except Exception as e:
                pass
    #print(row[4]) # <class 'str'> # auth score
    #print(row[5]) # <class 'str'> # hub score
    #count += 1
    node.append(row[0])
    node.append(row[1])
    node.append(parents_list)
    node.append(children_list)
    all_node.append(node)
with open("Graph_self reference remove(0308).csv", 'w', newline='', encoding='UTF-8') as file:
    writer = csv.writer(file, quoting=csv.QUOTE_ALL,delimiter=';')
    writer.writerows(all_node)

In [115]:
# 檢查
print(all_node[:10])

[['67', 'https://', ['https://www.snopes.com/about/', 'https://www.forane.com/zh/', 'https://www.forane.com/zh/', 'https://www.snopes.com/contact/', 'https://www.sartomer.com/zh/', 'https://www.sartomer.com/zh/', 'https://inmedase.org', 'https://equitablefutures.iftf.org/in-the-media/', 'https://www.gov.ie/en/service/be74d3-covid-19-pandemic-unemployment-payment', 'https://pages.nokia.com/T006V7-High-Accuracy-Indoor-Positioning-Digitalizing-assets-at-the-Chennai-Factory.html', 'https://pages.nokia.com/T006V7-High-Accuracy-Indoor-Positioning-Digitalizing-assets-at-the-Chennai-Factory.html', 'https://pages.nokia.com/T006V7-High-Accuracy-Indoor-Positioning-Digitalizing-assets-at-the-Chennai-Factory.html', 'https://getridhow.com', 'https://getridhow.com', 'https://www.sartomer.com/ja/', 'https://www.sartomer.com/ja/'], []], ['68', 'https:// https://careers-hackernoon.paperform.co/', ['https://careers.hackernoon.com'], []], ['69', 'https:// www.adition.com/kontakt/impressum/', ['https://www

In [116]:
# 開始實作

In [117]:
# 這邊需要建立 Sorted_All_list
All_list = []
with open("Graph_self reference remove(0308).csv", newline='', encoding = "utf-8") as csvfile:
    rows = csv.reader(csvfile, delimiter=';')
    for row in rows:
        All_list.append(row[1])
        for url in row[2]:
            All_list.append(url)
        for url2 in row[3]:
            All_list.append(url2)
All_list = set(All_list)
Sorted_All_list = sorted(All_list)

In [118]:
HITS = nx.DiGraph() # 建立一個叫HITS的有向圖

In [119]:
# 建立所有點的圖 
csv.field_size_limit(500 * 1024 * 1024)
with open("Graph_self reference remove(0308).csv", newline='', encoding = "utf-8") as csvfile:
    # 讀取 CSV 檔案內容
    rows = csv.reader(csvfile, delimiter=';') # 設定分隔號
    # 以迴圈檢查檢查前11筆
    #count = 0,
    for row in rows:
        #if count == 10000:  \n",
        #    break\n",
        # print(row)\n",
        # print(type(row)) # <class 'list'>\n",
        #print(row[1]) # <class 'str'> # original url\n",
        #print(row[2]) # <class 'str'> # parent list\n",
        #print(row[3]) # <class 'str'> # children list\n",
        #print(row[4]) # <class 'str'> # auth score\n",
        #print(row[5]) # <class 'str'> # hub score\n",
        HITS.add_node(int(row[0]) ,id = int(row[0]) ,original_url = row[1])
        #count += 1,

In [120]:
start_time = time.time()
# 開始建立關係
csv.field_size_limit(500 * 1024 * 1024)
with open("Graph_self reference remove(0308).csv", newline='', encoding = "utf-8") as csvfile: # 開啟之前做的圖檔
    # 讀取 CSV 檔案內容
    #count = 0
    rows = csv.reader(csvfile, delimiter=';') # 設定分隔號
    for row in rows:
        #if count == 100000:
        #    break
        children_list = str_to_url(row[3])
        if len(children_list) != 0: #out_link集合不為0才做
            for out_link in children_list:
                index = binary_search_for_nx(Sorted_All_list, out_link)
                # print(binary_search_for_nx(all_original_list, row[1]))
                if index != "Node Not Found!":
                    #print(row[0]) #row[0] <class: str>
                    original_id = int(row[0])
                    HITS.add_edge(original_id, index)  # type int, type int
                    # 上面一行加入的點為 原始的id 跟 index
                    # 注意，這一行如果新增的點不再圖中，會自動新增   
        #count += 1
print("花費:" + str(float(time.time() - start_time)) + "秒")

花費:53.798524379730225秒


In [121]:
binary_search_for_nx(Sorted_All_list, 'https://xn--9i1b14l89cjqd521a.money-sang.co.kr/%EA%B3%A8%EB%93%9C%EC%9C%99%EB%A8%B8%EB%8B%88%EC%83%81/')

1742652

# HITS

In [122]:
help(nx.hits)

Help on function hits in module networkx.algorithms.link_analysis.hits_alg:

hits(G, max_iter=100, tol=1e-08, nstart=None, normalized=True)
    Returns HITS hubs and authorities values for nodes.
    
    The HITS algorithm computes two numbers for a node.
    Authorities estimates the node value based on the incoming links.
    Hubs estimates the node value based on outgoing links.
    
    Parameters
    ----------
    G : graph
      A NetworkX graph
    
    max_iter : integer, optional
      Maximum number of iterations in power method.
    
    tol : float, optional
      Error tolerance used to check convergence in power method iteration.
    
    nstart : dictionary, optional
      Starting value of each node for power method iteration.
    
    normalized : bool (default=True)
       Normalize results by the sum of all of the values.
    
    Returns
    -------
    (hubs,authorities) : two-tuple of dictionaries
       Two dictionaries keyed by node containing the hub and auth

In [137]:
start_time = time.time()
h,a=nx.hits(HITS, max_iter=100 , normalized=False) # 得到兩個dictionary(未排序)
print("花費:" + str(float(time.time() - start_time)) + "秒")

花費:11.746010065078735秒


In [138]:
help(nx.hits)

Help on function hits in module networkx.algorithms.link_analysis.hits_alg:

hits(G, max_iter=100, tol=1e-08, nstart=None, normalized=True)
    Returns HITS hubs and authorities values for nodes.
    
    The HITS algorithm computes two numbers for a node.
    Authorities estimates the node value based on the incoming links.
    Hubs estimates the node value based on outgoing links.
    
    Parameters
    ----------
    G : graph
      A NetworkX graph
    
    max_iter : integer, optional
      Maximum number of iterations in power method.
    
    tol : float, optional
      Error tolerance used to check convergence in power method iteration.
    
    nstart : dictionary, optional
      Starting value of each node for power method iteration.
    
    normalized : bool (default=True)
       Normalize results by the sum of all of the values.
    
    Returns
    -------
    (hubs,authorities) : two-tuple of dictionaries
       Two dictionaries keyed by node containing the hub and auth

In [139]:
HITS.nodes[1294390]

{'id': 1294390, 'original_url': 'https://www.flipboard.com/@historychannel'}

In [140]:
top_n_authority_for_nx(a, HITS, 1000)

Top 1 {'id': 136442, 'original_url': 'https://cals.ncsu.edu/applied-ecology/news/striperhub-will-address-seafood-deficit/'}
weight: 5.921053332514803e-17
Top 2 {'id': 136446, 'original_url': 'https://cals.ncsu.edu/crop-and-soil-sciences/about/'}
weight: 5.918576843032559e-17
Top 3 {'id': 319466, 'original_url': 'https://extensions.joomla.org'}
weight: 5.916078059432497e-17
Top 4 {'id': 361836, 'original_url': 'https://gardenista.hu/tag/kert/'}
weight: 5.916078059432497e-17
Top 5 {'id': 413164, 'original_url': 'https://hortnews.extension.iastate.edu/2020/09/richard-jauron-retires'}
weight: 5.916078059432497e-17
Top 6 {'id': 512955, 'original_url': 'https://localgiftcards.com/'}
weight: 5.916078059432497e-17
Top 7 {'id': 577682, 'original_url': 'https://ncdc.am/coronavirus/confirmed-cases-by-days/'}
weight: 5.916078059432497e-17
Top 8 {'id': 582234, 'original_url': 'https://newdealforhighered.org/contact-us/'}
weight: 5.916078059432497e-17
Top 9 {'id': 1167876, 'original_url': 'https://w

weight: 1.500218090797759e-17
Top 867 {'id': 396291, 'original_url': 'https://happyfamilydrugstore.com/isoptin'}
weight: 1.500218090797759e-17
Top 868 {'id': 396292, 'original_url': 'https://happyfamilydrugstore.com/isoptin-sr'}
weight: 1.500218090797759e-17
Top 869 {'id': 396293, 'original_url': 'https://happyfamilydrugstore.com/isordil'}
weight: 1.500218090797759e-17
Top 870 {'id': 396294, 'original_url': 'https://happyfamilydrugstore.com/januvia'}
weight: 1.500218090797759e-17
Top 871 {'id': 396295, 'original_url': 'https://happyfamilydrugstore.com/kaletra'}
weight: 1.500218090797759e-17
Top 872 {'id': 396296, 'original_url': 'https://happyfamilydrugstore.com/kamagra'}
weight: 1.500218090797759e-17
Top 873 {'id': 396297, 'original_url': 'https://happyfamilydrugstore.com/kamagra-effervescent'}
weight: 1.500218090797759e-17
Top 874 {'id': 396298, 'original_url': 'https://happyfamilydrugstore.com/kamagra-gold'}
weight: 1.500218090797759e-17
Top 875 {'id': 396299, 'original_url': 'https

In [141]:
top_n_hub_for_nx(h, HITS, 100)

Top 1 {'id': 851799, 'original_url': 'https://surry.ces.ncsu.edu/2021/12/2022-private-pesticide-applicator-training-opportunities/'}
weight: 1.0899320973436116e-14
Top 2 {'id': 851786, 'original_url': 'https://surry.ces.ncsu.edu'}
weight: 1.0895445630861967e-14
Top 3 {'id': 523076, 'original_url': 'https://madison.ces.ncsu.edu'}
weight: 1.0895326433497961e-14
Top 4 {'id': 146584, 'original_url': 'https://catawba.ces.ncsu.edu'}
weight: 1.0895026795829569e-14
Top 5 {'id': 144076, 'original_url': 'https://carteret.ces.ncsu.edu'}
weight: 1.0891660093474906e-14
Top 6 {'id': 391832, 'original_url': 'https://guilford.ces.ncsu.edu'}
weight: 1.0889489311076207e-14
Top 7 {'id': 212389, 'original_url': 'https://davidson.ces.ncsu.edu'}
weight: 1.0885953158980553e-14
Top 8 {'id': 459773, 'original_url': 'https://jackson.ces.ncsu.edu'}
weight: 1.0883612157377596e-14
Top 9 {'id': 616597, 'original_url': 'https://onslow.ces.ncsu.edu'}
weight: 1.088236192111941e-14
Top 10 {'id': 501329, 'original_url':

# PageRank

In [128]:
help(nx.pagerank)

Help on function pagerank in module networkx.algorithms.link_analysis.pagerank_alg:

pagerank(G, alpha=0.85, personalization=None, max_iter=100, tol=1e-06, nstart=None, weight='weight', dangling=None)
    Returns the PageRank of the nodes in the graph.
    
    PageRank computes a ranking of the nodes in the graph G based on
    the structure of the incoming links. It was originally designed as
    an algorithm to rank web pages.
    
    Parameters
    ----------
    G : graph
      A NetworkX graph.  Undirected graphs will be converted to a directed
      graph with two directed edges for each undirected edge.
    
    alpha : float, optional
      Damping parameter for PageRank, default=0.85.
    
    personalization: dict, optional
      The "personalization vector" consisting of a dictionary with a
      key some subset of graph nodes and personalization value each of those.
      At least one personalization value must be non-zero.
      If not specfiied, a nodes personalization 

In [129]:
pagerank = nx.pagerank(HITS, alpha= 0.9)

In [130]:
top_n_pagerank_for_nx(pagerank, HITS, 100)

Top 1 {'id': 1179125, 'original_url': 'https://www.cloudflare.com/analysts/'}
weight: 0.0005310132187743104
Top 2 {'id': 1451681, 'original_url': 'https://www.mediawiki.org/w/index.php?title=Help:Contents&oldid=4979264'}
weight: 5.920961227281714e-05
Top 3 {'id': 1034204, 'original_url': 'https://wikimediafoundation.org/2018/09/30/international-translation-day/'}
weight: 5.58816069362734e-05
Top 4 {'id': 1451942, 'original_url': 'https://www.mediawiki.org/wiki/Special:MyLanguage/Manual:What_is_MediaWiki%3F'}
weight: 5.480298570989762e-05
Top 5 {'id': 345697, 'original_url': 'https://foundation.wikimedia.org/wiki/Donazioni'}
weight: 5.300608782012687e-05
Top 6 {'id': 345736, 'original_url': 'https://foundation.wikimedia.org/wiki/Project:About'}
weight: 4.323927771805603e-05
Top 7 {'id': 1179127, 'original_url': 'https://www.cloudflare.com/apps'}
weight: 3.473972922834174e-05
Top 8 {'id': 971834, 'original_url': 'https://twitter.com/privacypros'}
weight: 2.346878463593308e-05
Top 9 {'id'